<a href="https://colab.research.google.com/github/MayankDey20/Automated-Security-Log-Analyzer/blob/main/Automated_Security_Log_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# pip install gradio

import gradio as gr
import re
from collections import Counter

def analyze_logs(log_data):
    """
    Analyzes web server access logs for suspicious activity.
    """
    if not log_data or not log_data.strip():
        raise gr.Error("Please paste log data into the text box.")

    # Regex to parse a common web server log format:
    # IP - - [Date] "METHOD /path HTTP/version" STATUS_CODE ...
    log_pattern = re.compile(r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}) .*? "(\S+ .*? \S+)" (\d{3})')

    ip_counter = Counter()
    error_ips = Counter()
    suspicious_requests = {}

    findings = []

    for line in log_data.strip().split('\n'):
        match = log_pattern.match(line)
        if not match:
            continue

        ip, request, status_code = match.groups()
        status_code = int(status_code)

        ip_counter[ip] += 1

        # --- Rule 1: Look for multiple client errors (4xx) ---
        if 400 <= status_code < 500:
            error_ips[ip] += 1

        # --- Rule 2: Look for suspicious URL patterns (scanners, traversal) ---
        suspicious_patterns = {
            "Directory Traversal": r'\.\./',
            "SQL Injection Attempt": r'union|select|insert|--|;',
            "Common Scanner Path": r'wp-login|admin|phpmyadmin',
            "Sensitive File Access": r'/etc/passwd|/.git|/.env'
        }
        for name, pattern in suspicious_patterns.items():
            if re.search(pattern, request, re.IGNORECASE):
                if ip not in suspicious_requests:
                    suspicious_requests[ip] = []
                # Avoid duplicate pattern reports for the same IP
                if name not in [item[0] for item in suspicious_requests[ip]]:
                    suspicious_requests[ip].append((name, request))


    # --- Generate Report ---

    # Analyze IPs with high error counts
    high_error_offenders = []
    for ip, count in error_ips.items():
        if count > 5: # Threshold for high error count
            risk_score = 10 + count * 2
            high_error_offenders.append({
                "ip": ip,
                "count": count,
                "total_requests": ip_counter[ip],
                "risk_score": risk_score
            })

    if high_error_offenders:
        report_section = "### 🔴 High Error Rate IPs\n\nThese IPs have generated an unusually high number of client-side errors (e.g., 404 Not Found, 403 Forbidden)."
        for offender in sorted(high_error_offenders, key=lambda x: x['risk_score'], reverse=True):
            report_section += f"\n- **IP:** `{offender['ip']}`\n  - **Risk Score:** {offender['risk_score']}\n  - **Error Count:** {offender['count']} (out of {offender['total_requests']} total requests)"
        findings.append(report_section)

    # Analyze IPs with suspicious requests
    suspicious_pattern_offenders = []
    for ip, details in suspicious_requests.items():
        risk_score = 15 * len(details) # Higher score for more types of suspicious activity
        suspicious_pattern_offenders.append({
            "ip": ip,
            "details": details,
            "risk_score": risk_score
        })

    if suspicious_pattern_offenders:
        report_section = "### 🟡 Suspicious URL Patterns Detected\n\nThese IPs have requested URLs matching common attack or scanning patterns."
        for offender in sorted(suspicious_pattern_offenders, key=lambda x: x['risk_score'], reverse=True):
            report_section += f"\n- **IP:** `{offender['ip']}`\n  - **Risk Score:** {offender['risk_score']}\n  - **Detected Patterns:**"
            for name, request in offender['details']:
                 report_section += f"\n    - `{name}` (Request: `{request}`)"
        findings.append(report_section)

    if not findings:
        return "## ✅ Analysis Complete\n\nNo major suspicious activities found based on the defined rules."

    final_report = "## 🛡️ Security Log Analysis Report\n\n---\n\n" + "\n\n---\n\n".join(findings)
    return final_report


# --- Example Log Data ---
example_log_1 = """
192.168.1.1 - - [10/Mar/2023:13:55:36 +0000] "GET /index.html HTTP/1.1" 200 2048
88.1.2.3 - - [10/Mar/2023:13:56:10 +0000] "GET /admin HTTP/1.1" 404 128
88.1.2.3 - - [10/Mar/2023:13:56:11 +0000] "GET /wp-login.php HTTP/1.1" 404 128
88.1.2.3 - - [10/Mar/2023:13:56:12 +0000] "GET /administrator/index.php HTTP/1.1" 404 128
88.1.2.3 - - [10/Mar/2023:13:56:13 +0000] "GET /phpmyadmin/ HTTP/1.1" 404 128
88.1.2.3 - - [10/Mar/2023:13:56:14 +0000] "GET /old-login HTTP/1.1" 404 128
88.1.2.3 - - [10/Mar/2023:13:56:15 +0000] "GET /backup/config.php HTTP/1.1" 404 128
10.0.0.5 - - [10/Mar/2023:13:57:00 +0000] "GET /images/logo.png HTTP/1.1" 200 5120
45.12.33.4 - - [10/Mar/2023:13:58:01 +0000] "GET /users.php?id=1' UNION SELECT 1,2,3 -- HTTP/1.1" 500 100
45.12.33.4 - - [10/Mar/2023:13:58:02 +0000] "GET /../../../../etc/passwd HTTP/1.1" 403 100
"""

# --- Gradio UI ---
with gr.Blocks(theme=gr.themes.Monochrome(), title="Log Analyzer") as app:
    gr.Markdown(
        """
        # 🛡️ Automated Security Log Analyzer
        Paste your raw web server access logs below to automatically scan for suspicious patterns.
        """
    )
    log_input = gr.Textbox(
        lines=15,
        label="Log Data",
        placeholder="Paste log file contents here...",
        value=example_log_1
    )
    analyze_btn = gr.Button("Analyze Logs", variant="primary")

    gr.Markdown("---")

    analysis_output = gr.Markdown(label="Analysis Report")

    analyze_btn.click(
        fn=analyze_logs,
        inputs=log_input,
        outputs=analysis_output
    )

if __name__ == "__main__":
    app.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e17d769c8288562005.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
